# Импорт необходимых библиотек
Импортируйте необходимые библиотеки, включая PyTorch.

In [9]:
# Импорт необходимых библиотек
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import ssl

# Загрузка и предварительная обработка датасета cifar10
Загрузите и предварительно обработайте датасет cifar10, используя torchvision.

In [10]:
# Загрузка и предварительная обработка датасета cifar10

# Преобразования, которые будут применены к изображениям
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# фикс для SSL
ssl._create_default_https_context = ssl._create_unverified_context

# Загрузка обучающего набора данных
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

# Загрузка тестового набора данных
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

# Классы в датасете cifar10
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


# Определение архитектуры сверточной нейронной сети
Определите архитектуру сверточной нейронной сети, используя PyTorch.

In [11]:
# Определение архитектуры сверточной нейронной сети
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Сверточный слой (3 входных канала, 6 выходных каналов, ядро свертки 5x5)
        self.conv1 = nn.Conv2d(3, 6, 5)
        # Слой пулинга (2x2)
        self.pool = nn.MaxPool2d(2, 2)
        # Сверточный слой (6 входных каналов, 16 выходных каналов, ядро свертки 5x5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # Полносвязный слой (16 * 5 * 5 входных нейронов, 120 выходных нейронов)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        # Полносвязный слой (120 входных нейронов, 84 выходных нейрона)
        self.fc2 = nn.Linear(120, 84)
        # Полносвязный слой (84 входных нейрона, 10 выходных нейронов)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Применяем свертку и пулинг
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # Преобразуем тензор в вектор
        x = x.view(-1, 16 * 5 * 5)
        # Применяем полносвязные слои
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Создаем экземпляр сети
net = Net()

# Определение функции потерь и оптимизатора
Определите функцию потерь и оптимизатор, используя PyTorch.

In [12]:
# Определение функции потерь и оптимизатора
# Используем кросс-энтропийную потерю
criterion = nn.CrossEntropyLoss()

# Используем стохастический градиентный спуск для оптимизации
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Обучение модели
Обучите модель, используя обучающий набор данных.

In [13]:
# Обучение модели
for epoch in range(2):  # Проходим по датасету несколько раз

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Получаем входные данные; data - это список [inputs, labels]
        inputs, labels = data

        # Обнуляем градиенты параметров
        optimizer.zero_grad()

        # Вперед + назад + оптимизация
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Печатаем статистику
        running_loss += loss.item()
        if i % 2000 == 1999:    # Печатаем каждые 2000 мини-пакетов
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.197
[1,  4000] loss: 1.878
[1,  6000] loss: 1.696
[1,  8000] loss: 1.607
[1, 10000] loss: 1.541
[1, 12000] loss: 1.492
[2,  2000] loss: 1.432
[2,  4000] loss: 1.397
[2,  6000] loss: 1.379
[2,  8000] loss: 1.349
[2, 10000] loss: 1.322
[2, 12000] loss: 1.293
Finished Training


# Тестирование модели
Протестируйте модель на тестовом наборе данных и оцените ее производительность.

In [14]:
# Импорт необходимых библиотек
import torch

# Переводим модель в режим оценки
net.eval()

# Инициализируем счетчики правильных и общего числа предсказаний
correct = 0
total = 0

# Отключаем вычисление градиентов, так как они не нужны в процессе тестирования
with torch.no_grad():
    for data in testloader:
        # Получаем изображения и метки
        images, labels = data
        # Прогоняем изображения через сеть и получаем предсказания
        outputs = net(images)
        # Получаем предсказанные классы
        _, predicted = torch.max(outputs.data, 1)
        # Обновляем счетчики
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Выводим процент правильных предсказаний
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 53 %
